# 提示模板
## 概述
本教學涵蓋如何使用 ```LangChain``` 建立和使用提示模板。
提示模板對於生成動態且靈活的提示至關重要，能夠滿足各種使用情境，如對話歷史記錄、結構化輸出和專門查詢。
在本教學中，我們將探索建立 ```PromptTemplate``` 物件的方法、應用部分變數、透過 YAML 檔案管理模板，以及利用進階工具如 ```ChatPromptTemplate``` 和 ```MessagePlaceholder``` 來增強功能。

### 目錄
- [概述](#概述)
- [環境設定](#環境設定)
- [建立 PromptTemplate 物件](#建立prompttemplate物件)
- [使用 partial_variables](#使用partial_variables)
- [從 YAML 檔案載入提示模板](#從yaml檔案載入提示模板)
- [ChatPromptTemplate](#chatprompttemplate)
- [MessagePlaceholder](#messageplaceholder)

### 參考資料
- [LangChain Documentation : Prompts](https://python.langchain.com/api_reference/core/prompts.html#)

---

## 我的見解

提示模板是 LangChain 生態系統的基礎組件，它讓開發者能夠建立可重用、動態的提示，大幅提高了 AI 應用開發的效率和一致性。

## 學習補充重點

**提示模板的核心價值：**
- **可重用性**：一次建立，多處使用，減少重複程式碼
- **動態性**：支援變數替換，適應不同的輸入情境
- **結構化**：清晰的模板結構便於維護和除錯
- **標準化**：確保整個應用中提示的一致性

**主要組件功能：**

**PromptTemplate：**
- 基礎的文字模板，支援變數插值
- 適合簡單的單輪對話或查詢場景
- 提供輸入驗證和格式化功能

**partial_variables：**
- 允許預設部分變數值
- 適用於有固定元素的模板（如系統資訊、日期等）
- 提高模板的靈活性和可配置性

**YAML 檔案管理：**
- 將提示邏輯與程式碼分離
- 便於非技術人員編輯和維護
- 支援版本控制和團隊協作

**ChatPromptTemplate：**
- 專為多輪對話設計
- 支援系統、使用者、助手等不同角色
- 更適合複雜的對話式 AI 應用

**MessagePlaceholder：**
- 動態插入訊息序列
- 特別適合處理對話歷史記錄
- 支援靈活的訊息結構

**實際應用場景：**
- **客服機器人**：標準化的問答模板
- **內容生成**：部落格文章、產品描述等格式化內容
- **資料分析**：結構化的分析報告模板
- **教育應用**：個人化的學習內容生成

**最佳實務建議：**
- 設計模板時考慮變數的命名規範
- 使用部分變數來設定常用的預設值
- 將複雜的提示分解為可組合的子模板
- 定期審查和優化模板的效果

**進階技巧：**
- 結合條件邏輯建立動態模板
- 使用模板繼承減少重複程式碼
- 實施模板版本管理和 A/B 測試
- 建立模板庫以促進團隊共享

提示模板是構建可維護、可擴展 AI 應用的關鍵工具，掌握其使用方法對於 LangChain 開發者來說至關重要。

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials.
- You can check out the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [ ]:
%%capture --no-stderr
%pip install langchain-opentutorial


[notice] A new release of pip is available: 24.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_community",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)

In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [ ]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        # "OPENAI_API_KEY": "",
        # "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "Prompt-Template",
    }
)

Environment variables have been set successfully.


Let's setup ```ChatOpenAI``` with ```gpt-4o``` model.

In [ ]:
from langchain_openai import ChatOpenAI

# Load the model
llm = ChatOpenAI(model_name="gpt-4o")

## 建立 ```PromptTemplate``` 物件
有兩種方式可以建立 ```PromptTemplate``` 物件。
- 1. 使用 ```from_template()``` 方法
- 2. 一次性建立 ```PromptTemplate``` 物件和提示

---

## 我的見解

這兩種建立方法提供了不同層次的靈活性，開發者可以根據具體需求選擇最適合的方式。

## 學習補充重點

**方法一：```from_template()``` 方法**
- **優點**：簡潔快速，適合簡單的模板建立
- **適用場景**：原型開發、簡單應用、快速測試
- **特點**：自動推斷變數，減少配置工作

**範例應用：**
```python
from langchain.prompts import PromptTemplate

# 簡單直接的方式
template = "告訴我關於 {topic} 的 {details}"
prompt = PromptTemplate.from_template(template)
```

**方法二：完整物件建立**
- **優點**：完全控制，支援進階配置
- **適用場景**：複雜應用、需要詳細配置的模板
- **特點**：明確指定輸入變數、輸出解析器等

**範例應用：**
```python
# 完整配置的方式
prompt = PromptTemplate(
    input_variables=["topic", "details"],
    template="告訴我關於 {topic} 的 {details}",
    # 可以添加更多配置選項
    validate_template=True
)
```

**選擇建議：**
- **快速原型**：使用 ```from_template()```
- **生產環境**：考慮完整物件建立以獲得更好的控制
- **團隊開發**：明確的變數定義有助於程式碼可讀性
- **複雜邏輯**：需要額外配置時選擇完整建立方式

**進階技巧：**
- 結合兩種方法的優勢
- 使用工廠模式管理不同類型的模板
- 實施模板驗證和錯誤處理機制

### Method 1. Using the ```from_template()``` method

- Define template with variable as ```{variable}``` .

In [ ]:
from langchain_core.prompts import PromptTemplate

# Define template. In this case, {country} is a variable
template = "What is the capital of {country}?"

# Create a `PromptTemplate` object using the `from_template` method
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}?')

You can complete the prompt by assigning a value to the variable ```country``` .

In [ ]:
# Create prompt. Assign value to the variable using `format` method
prompt = prompt.format(country="United States of America")
prompt

'What is the capital of United States of America?'

In [ ]:
# Define template
template = "What is the capital of {country}?"

# Create a `PromptTemplate` object using the `from_template` method
prompt = PromptTemplate.from_template(template)

# Create chain
chain = prompt | llm

In [ ]:
# Replace the country variable with a value of your choice
chain.invoke("United States of America").content

'The capital of the United States of America is Washington, D.C.'

### 方法二：一次性建立 ```PromptTemplate``` 物件和提示
明確指定 ```input_variables``` 以進行額外驗證。
否則，這些變數與模板字串中的變數不匹配可能會在實例化時引發異常。

---

## 我的見解

明確指定 ```input_variables``` 是一個重要的最佳實務，它提供了額外的安全性和可讀性，特別是在複雜的生產環境中。

## 學習補充重點

**```input_variables``` 的重要性：**

**驗證機制：**
- **變數一致性檢查**：確保模板中的變數與宣告的一致
- **錯誤提前發現**：在實例化階段就捕獲問題，而非執行時
- **型別安全**：提供更好的開發時檢查

**範例說明：**
```python
# 正確的做法
prompt = PromptTemplate(
    input_variables=["topic", "level"],
    template="解釋 {topic} 給 {level} 程度的學習者"
)

# 錯誤示範 - 變數不匹配
prompt = PromptTemplate(
    input_variables=["topic"],  # 遺漏了 'level'
    template="解釋 {topic} 給 {level} 程度的學習者"
    # 這會引發 ValueError
)
```

**常見錯誤類型：**
- **遺漏變數**：模板中有變數但未在 input_variables 中宣告
- **多餘變數**：input_variables 中有變數但模板中未使用
- **變數名稱錯誤**：拼寫錯誤導致的不匹配

**最佳實務建議：**
- **始終明確宣告**：即使 LangChain 可以自動推斷
- **使用描述性名稱**：讓變數用途一目了然
- **文檔化複雜變數**：為複雜的變數添加註釋
- **單元測試**：測試模板的各種輸入組合

**進階配置選項：**
```python
prompt = PromptTemplate(
    input_variables=["topic", "level"],
    template="解釋 {topic} 給 {level} 程度的學習者",
    validate_template=True,  # 啟用嚴格驗證
    template_format="f-string"  # 指定格式
)
```

**除錯技巧：**
- 檢查變數名稱的拼寫
- 確認大括號語法正確
- 使用 IDE 的語法高亮協助檢查
- 建立測試案例驗證模板行為

這種明確的方法雖然需要更多程式碼，但能大幅提高程式碼的可靠性和可維護性。

In [ ]:
# Define template
template = "What is the capital of {country}?"

# Create a prompt template with `PromptTemplate` object
prompt = PromptTemplate(
    template=template,
    input_variables=["country"],
)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}?')

In [ ]:
# Create prompt
prompt.format(country="United States of America")

'What is the capital of United States of America?'

In [ ]:
# Define template
template = "What are the capitals of {country1} and {country2}, respectively?"

# Create a prompt template with `PromptTemplate` object
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "United States of America"  # Pass `partial_variables` in dictionary form
    },
)
prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': 'United States of America'}, template='What are the capitals of {country1} and {country2}, respectively?')

In [ ]:
prompt.format(country1="South Korea")

'What are the capitals of South Korea and United States of America, respectively?'

In [ ]:
prompt_partial = prompt.partial(country2="India")
prompt_partial

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': 'India'}, template='What are the capitals of {country1} and {country2}, respectively?')

In [ ]:
prompt_partial.format(country1="South Korea")

'What are the capitals of South Korea and India, respectively?'

In [ ]:
chain = prompt_partial | llm

In [ ]:
chain.invoke("United States of America").content

'The capital of the United States of America is Washington, D.C., and the capital of India is New Delhi.'

In [ ]:
chain.invoke({"country1": "United States of America", "country2": "India"}).content

'The capital of the United States of America is Washington, D.C., and the capital of India is New Delhi.'

## 使用 ```partial_variables```
使用 ```partial_variables```，您可以部分應用函式。這在有**共用變數**需要分享時特別有用。
常見的範例是**日期或時間**。
假設您想在提示中指定當前日期，將日期硬編碼到提示中或與其他輸入變數一起傳遞可能不太實用。在這種情況下，使用返回當前日期的函式來部分修改提示會更加方便。

---

## 我的見解

```partial_variables``` 是提示模板中非常實用的功能，它解決了動態值（如時間、系統資訊）與靜態模板結合的問題，大幅提升了模板的靈活性和實用性。

## 學習補充重點

**```partial_variables``` 的核心價值：**

**動態內容處理：**
- **時間資訊**：當前日期、時間戳、時區資訊
- **系統資訊**：用戶 ID、會話 ID、應用版本
- **上下文資訊**：地理位置、語言設定、權限級別

**使用方式：**
```python
from datetime import datetime
from langchain.prompts import PromptTemplate

def get_current_date():
    return datetime.now().strftime("%Y-%m-%d")

# 使用函式作為部分變數
prompt = PromptTemplate(
    input_variables=["task"],
    template="今天是 {date}，請幫我完成以下任務：{task}",
    partial_variables={"date": get_current_date}
)

# 或者使用固定值
prompt = PromptTemplate(
    input_variables=["query"],
    template="系統版本：{version}\n用戶查詢：{query}",
    partial_variables={"version": "v2.1.0"}
)
```

**實際應用場景：**

**商業應用：**
- **報告生成**：自動插入報告日期和時間
- **個人化內容**：根據用戶資訊客製化訊息
- **合規要求**：自動記錄操作時間和版本資訊

**技術應用：**
- **日誌記錄**：統一的時間戳格式
- **API 文檔**：動態插入當前 API 版本
- **測試環境**：區分不同環境的配置資訊

**進階技巧：**

**條件式部分變數：**
```python
def get_greeting():
    hour = datetime.now().hour
    if hour < 12:
        return "早安"
    elif hour < 18:
        return "午安"
    else:
        return "晚安"

prompt = PromptTemplate(
    input_variables=["name"],
    template="{greeting}，{name}！有什麼我可以幫助您的嗎？",
    partial_variables={"greeting": get_greeting}
)
```

**組合多個部分變數：**
```python
prompt = PromptTemplate(
    input_variables=["question"],
    template="""
    時間：{timestamp}
    系統：{system_info}
    用戶問題：{question}
    
    請根據上述資訊提供回答。
    """,
    partial_variables={
        "timestamp": lambda: datetime.now().isoformat(),
        "system_info": lambda: "AI助手 v3.0"
    }
)
```

**最佳實務建議：**
- **快取機制**：對於成本較高的函式，考慮添加快取
- **錯誤處理**：確保部分變數函式的健壯性
- **性能考量**：避免在部分變數中執行耗時操作
- **測試覆蓋**：為部分變數函式建立單元測試

**常見陷阱與解決方案：**
- **時區問題**：確保時間函式使用正確的時區
- **格式一致性**：統一日期時間格式
- **函式純度**：避免副作用，確保函式的可預測性

```partial_variables``` 讓模板變得更加動態和智能，是建立專業級 AI 應用的重要工具。

In [ ]:
from datetime import datetime

# Print the current date
datetime.now().strftime("%B %d")

'January 14'

In [ ]:
# Define function to return the current date
def get_today():
    return datetime.now().strftime("%B %d")

In [ ]:
prompt = PromptTemplate(
    template="Today's date is {today}. Please list {n} celebrities whose birthday is today. Please specify their date of birth.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  # Pass `partial_variables` in dictionary form
    },
)

In [ ]:
# Create prompt
prompt.format(n=3)

"Today's date is January 14. Please list 3 celebrities whose birthday is today. Please specify their date of birth."

In [ ]:
# Create chain
chain = prompt | llm

In [ ]:
# Invoke chain and check the result
print(chain.invoke(3).content)

Here are three celebrities born on January 14:

1. **Dave Grohl** - Born on January 14, 1969.
2. **LL Cool J** - Born on January 14, 1968.
3. **Jason Bateman** - Born on January 14, 1969.


In [ ]:
# Invoke chain and check the result
print(chain.invoke({"today": "Jan 02", "n": 3}).content)

Here are three celebrities born on January 2:

1. **Cuba Gooding Jr.** - Born on January 2, 1968.
2. **Taye Diggs** - Born on January 2, 1971.
3. **Kate Bosworth** - Born on January 2, 1983.


## Load Prompt Templates from YAML Files

You can manage prompt templates in seperate yaml files and load using ```load_prompt``` .

In [ ]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml", encoding="utf-8")
prompt

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='What is the color of {fruit}?')

In [ ]:
prompt.format(fruit="an apple")

'What is the color of an apple?'

In [ ]:
prompt2 = load_prompt("prompts/capital.yaml")
print(prompt2.format(country="United States of America"))

Please provide information about the capital city of United States of America.
Summarize the characteristics of the capital in the following format, within 300 words.
----
[Format]
1. Area
2. Population
3. Historical Sites
4. Regional Products

#Answer:



## ```ChatPromptTemplate```
```ChatPromptTemplate``` 可用於將對話歷史記錄包含為提示。
訊息以元組格式 (```role```, ```message```) 結構化，並建立為清單。
**角色**
- ```system```：系統設定訊息，通常用於全域設定相關的提示。
- ```human```：使用者輸入訊息。
- ```ai```：AI 回應訊息。

---

## 我的見解

```ChatPromptTemplate``` 是處理多輪對話的強大工具，它提供了結構化的方式來管理不同角色的訊息，這對於建立自然且上下文感知的對話系統至關重要。

## 學習補充重點

**```ChatPromptTemplate``` 的核心優勢：**

**角色區分系統：**
- **清晰的對話結構**：明確區分不同參與者的訊息
- **上下文保持**：維持完整的對話歷史記錄
- **角色一致性**：確保每個角色的特性和行為一致

**三種核心角色詳解：**

**```system``` 角色：**
```python
from langchain.prompts import ChatPromptTemplate

# 系統角色設定 AI 的行為和個性
messages = [
    ("system", "你是一個專業的程式設計導師，請用清晰易懂的方式回答問題。"),
    ("human", "什麼是遞迴？"),
    ("ai", "遞迴是一種程式設計技巧..."),
    ("human", "可以給我一個範例嗎？")
]

chat_prompt = ChatPromptTemplate.from_messages(messages)
```

**```human``` 角色：**
- 代表使用者的輸入和查詢
- 可以包含問題、指令或對話內容
- 通常是觸發 AI 回應的起點

**```ai``` 角色：**
- 代表 AI 助手的回應
- 用於建立對話歷史記錄
- 提供上下文供後續互動參考

**實際應用模式：**

**客服對話系統：**
```python
chat_template = ChatPromptTemplate.from_messages([
    ("system", "你是一個客服代表，友善且專業地幫助客戶解決問題。"),
    ("human", "我的訂單還沒收到"),
    ("ai", "很抱歉聽到這個問題，讓我幫您查詢訂單狀態。"),
    ("human", "訂單號碼是 {order_number}")
])
```

**教育輔導系統：**
```python
education_template = ChatPromptTemplate.from_messages([
    ("system", "你是一位耐心的數學老師，用循序漸進的方式教學。"),
    ("human", "我不懂二次方程式"),
    ("ai", "沒關係，我們從基礎開始學習..."),
    ("human", "{new_question}")
])
```

**動態對話建構：**
```python
def build_chat_history(conversation_history, new_message):
    messages = [("system", "你是一個有用的助手")]
    
    # 添加歷史對話
    for turn in conversation_history:
        messages.append(("human", turn["user"]))
        messages.append(("ai", turn["assistant"]))
    
    # 添加新訊息
    messages.append(("human", new_message))
    
    return ChatPromptTemplate.from_messages(messages)
```

**進階技巧：**

**條件式訊息：**
```python
def create_conditional_chat(user_type, query):
    if user_type == "expert":
        system_msg = "請提供技術深度的專業回答"
    else:
        system_msg = "請用簡單易懂的方式回答"
    
    return ChatPromptTemplate.from_messages([
        ("system", system_msg),
        ("human", query)
    ])
```

**模板變數整合：**
```python
chat_template = ChatPromptTemplate.from_messages([
    ("system", "你是專精於 {domain} 的專家"),
    ("human", "關於 {topic} 的問題：{question}")
])

# 使用時填入變數
formatted = chat_template.format_messages(
    domain="機器學習",
    topic="神經網路",
    question="什麼是反向傳播？"
)
```

**最佳實務建議：**
- **角色一致性**：確保每個角色的語調和風格一致
- **適當的系統提示**：清楚定義 AI 的行為和限制
- **對話長度管理**：避免對話歷史過長影響效能
- **上下文相關性**：只保留與當前對話相關的歷史記錄

```ChatPromptTemplate``` 是建立高品質對話系統的基礎工具，它讓開發者能夠創建更自然、更有上下文感知能力的 AI 互動體驗。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("What is the capital of {country}?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}?'), additional_kwargs={})])

In [ ]:
chat_prompt.format(country="United States of America")

'Human: What is the capital of United States of America?'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "You are a friendly AI assistant. Your name is {name}."),
        ("human", "Nice to meet you!"),
        ("ai", "Hello! How can I assist you?"),
        ("human", "{user_input}"),
    ]
)

# Create chat messages
messages = chat_template.format_messages(name="Teddy", user_input="What is your name?")
messages

[SystemMessage(content='You are a friendly AI assistant. Your name is Teddy.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Nice to meet you!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello! How can I assist you?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]

You can directly invoke LLM using the messages created above.

In [ ]:
llm.invoke(messages).content

'My name is Teddy. How can I help you today?'

You can also create a chain to execute.

In [ ]:
chain = chat_template | llm

In [ ]:
chain.invoke({"name": "Teddy", "user_input": "What is your name?"}).content

'My name is Teddy. How can I help you today?'

## ```ChatPromptTemplate```
```ChatPromptTemplate``` 可用於將對話歷史記錄包含為提示。
訊息以元組格式 (```role```, ```message```) 結構化，並建立為清單。
**角色**
- ```system```：系統設定訊息，通常用於全域設定相關的提示。
- ```human```：使用者輸入訊息。
- ```ai```：AI 回應訊息。

---

## 我的見解

```ChatPromptTemplate``` 是處理多輪對話的強大工具，它提供了結構化的方式來管理不同角色的訊息，這對於建立自然且上下文感知的對話系統至關重要。

## 學習補充重點

**```ChatPromptTemplate``` 的核心優勢：**

**角色區分系統：**
- **清晰的對話結構**：明確區分不同參與者的訊息
- **上下文保持**：維持完整的對話歷史記錄
- **角色一致性**：確保每個角色的特性和行為一致

**三種核心角色詳解：**

**```system``` 角色：**
```python
from langchain.prompts import ChatPromptTemplate

# 系統角色設定 AI 的行為和個性
messages = [
    ("system", "你是一個專業的程式設計導師，請用清晰易懂的方式回答問題。"),
    ("human", "什麼是遞迴？"),
    ("ai", "遞迴是一種程式設計技巧..."),
    ("human", "可以給我一個範例嗎？")
]

chat_prompt = ChatPromptTemplate.from_messages(messages)
```

**```human``` 角色：**
- 代表使用者的輸入和查詢
- 可以包含問題、指令或對話內容
- 通常是觸發 AI 回應的起點

**```ai``` 角色：**
- 代表 AI 助手的回應
- 用於建立對話歷史記錄
- 提供上下文供後續互動參考

**實際應用模式：**

**客服對話系統：**
```python
chat_template = ChatPromptTemplate.from_messages([
    ("system", "你是一個客服代表，友善且專業地幫助客戶解決問題。"),
    ("human", "我的訂單還沒收到"),
    ("ai", "很抱歉聽到這個問題，讓我幫您查詢訂單狀態。"),
    ("human", "訂單號碼是 {order_number}")
])
```

**教育輔導系統：**
```python
education_template = ChatPromptTemplate.from_messages([
    ("system", "你是一位耐心的數學老師，用循序漸進的方式教學。"),
    ("human", "我不懂二次方程式"),
    ("ai", "沒關係，我們從基礎開始學習..."),
    ("human", "{new_question}")
])
```

**動態對話建構：**
```python
def build_chat_history(conversation_history, new_message):
    messages = [("system", "你是一個有用的助手")]
    
    # 添加歷史對話
    for turn in conversation_history:
        messages.append(("human", turn["user"]))
        messages.append(("ai", turn["assistant"]))
    
    # 添加新訊息
    messages.append(("human", new_message))
    
    return ChatPromptTemplate.from_messages(messages)
```

**進階技巧：**

**條件式訊息：**
```python
def create_conditional_chat(user_type, query):
    if user_type == "expert":
        system_msg = "請提供技術深度的專業回答"
    else:
        system_msg = "請用簡單易懂的方式回答"
    
    return ChatPromptTemplate.from_messages([
        ("system", system_msg),
        ("human", query)
    ])
```

**模板變數整合：**
```python
chat_template = ChatPromptTemplate.from_messages([
    ("system", "你是專精於 {domain} 的專家"),
    ("human", "關於 {topic} 的問題：{question}")
])

# 使用時填入變數
formatted = chat_template.format_messages(
    domain="機器學習",
    topic="神經網路",
    question="什麼是反向傳播？"
)
```

**最佳實務建議：**
- **角色一致性**：確保每個角色的語調和風格一致
- **適當的系統提示**：清楚定義 AI 的行為和限制
- **對話長度管理**：避免對話歷史過長影響效能
- **上下文相關性**：只保留與當前對話相關的歷史記錄

```ChatPromptTemplate``` 是建立高品質對話系統的基礎工具，它讓開發者能夠創建更自然、更有上下文感知能力的 AI 互動體驗。

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a summarization specialist AI assistant. Your mission is to summarize conversations using key points.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "Summarize the conversation so far in {word_count} words."),
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

You can use ```MessagesPlaceholder``` to add the conversation message list.

In [ ]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "Hello! I’m Teddy. Nice to meet you."),
        ("ai", "Nice to meet you! I look forward to working with you."),
    ],
)

print(formatted_chat_prompt)

System: You are a summarization specialist AI assistant. Your mission is to summarize conversations using key points.
Human: Hello! I’m Teddy. Nice to meet you.
AI: Nice to meet you! I look forward to working with you.
Human: Summarize the conversation so far in 5 words.


In [ ]:
# Create chain
chain = chat_prompt | llm | StrOutputParser()

In [ ]:
# Invoke chain and check the result
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "Hello! I'm Teddy. Nice to meet you.",
            ),
            ("ai", "Nice to meet you! I look forward to working with you."),
        ],
    }
)

'Teddy introduces himself, exchanges greetings.'